In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from scipy.signal import butter, filtfilt
import os
from detecta import detect_onset, detect_peaks

def butter_lowpass_filtfilt(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low')
    y = filtfilt(b, a, data)
    return y

player_height = {
    'bobae' : 165,
    'bbs' : 165,
}
player_weight = {
    'bobae' : 60,
    'bbs' : 60,
}

In [21]:
day = '20240321'
DIR = glob(f'Data/{day}/*.csv')[:1]
DIR

['Data/20240321/uplift_data_export_20240322063503.csv']

In [22]:
sr = 240
dt = 1/sr

FALSE_DF  = {}
for dir in DIR:
    df = pd.read_csv(dir)
    for session in df.sessionid.unique():
        d = df[df['sessionid'] == session].reset_index(drop=True)
        session_num = session.split('_')[-1]
        name = d['athlete_name'].values[0]
        height = player_height[name]
        weight = player_weight[name]
        
        os.makedirs(f"Data/{day}/processing/{name}",exist_ok=True)
        handedness = d['handedness'].values[0]
        y_jc_offset = d['left_ankle_jc_3d_y'].min()
        jc_y_cols = []
        for c in d.columns:
            if '3d_y' in c:
                jc_y_cols.append(c)
        d[jc_y_cols]  = d[jc_y_cols] - y_jc_offset
        wrist_jc = d[['left_wrist_jc_3d_x','left_wrist_jc_3d_y','left_wrist_jc_3d_z']].copy()
        for c in wrist_jc.columns:
            wrist_jc[c] = butter_lowpass_filtfilt(wrist_jc[c], 6, sr, 2)
        
        wrist_jc_vel = (wrist_jc - wrist_jc.shift(2)).shift(-1)/dt
        wrist_jc_vel_sum = np.sqrt(np.sum(wrist_jc_vel**2, axis=1))
        
        plot = False
        try:
            timing = detect_onset(wrist_jc_vel_sum, threshold=0.5, n_above = sr * 0.3, show=plot)
            address = timing[0][0]
            top = [i for i in detect_peaks(wrist_jc_vel_sum, mph=5, mpd=sr*0.1, show=plot,valley=True) if address < i][0]
            finish = timing[-1][-1]
            impact = [i for i in detect_peaks(wrist_jc_vel_sum, mph=5, mpd=sr*0.1, show=plot) if top < i][0] + 5
            
            if address < top < impact < finish:
                d['address'] = address
                d['top'] = top
                d['impact'] = impact
                d['finish'] = finish
                d['height'] = height
                d['weight'] = weight
                d = d.reset_index(drop=True)
                d.to_csv(f"Data/{day}/processing/{name}/{name}_{int(session_num):03d}.csv", index =False)
            
            else:
                FALSE_DF[name_trial] = d
                print(f"{name}_{int(session_num):03d} : 시점 오류") 
                

        except Exception as e:
            print(f"{name}_{int(session_num):03d} : {e}")
            name_trial = f"{name}_{int(session_num):03d}"
            FALSE_DF[name_trial] = d

In [26]:
markers.keys()

dict_keys(['nose_3d', 'left_eye_3d', 'right_eye_3d', 'left_ear_3d', 'right_ear_3d', 'left_shoulder_jc_3d', 'right_shoulder_jc_3d', 'left_elbow_jc_3d', 'right_elbow_jc_3d', 'left_wrist_jc_3d', 'right_wrist_jc_3d', 'left_hip_jc_3d', 'right_hip_jc_3d', 'left_knee_jc_3d', 'right_knee_jc_3d', 'left_ankle_jc_3d', 'right_ankle_jc_3d', 'right_heel_3d', 'left_heel_3d', 'proximal_neck_3d', 'pelvis_3d', 'mid_head_3d', 'right_mid_toe_3d', 'left_mid_toe_3d'])

In [25]:
import sys
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display
from Functions import Animated


speed = 1
name = ''
axis = 'y'
jc_cols = []
for c in d.columns:
    if '3d' in c:
        jc_cols.append(c) 
markers = {}
mrk_df = d[jc_cols].copy()
for i in range(0,len(mrk_df.columns), 3):
    markers[mrk_df.columns[i].replace('_x','')] = mrk_df.iloc[:,i:i+3].T.values

connections = [
    ('pelvis_3d','left_hip_jc_3d'),('pelvis_3d','right_hip_jc_3d'),
    ('proximal_neck_3d', 'right_shoulder_jc_3d'),('proximal_neck_3d', 'left_shoulder_jc_3d'),
    ('pelvis_3d','proximal_neck_3d'),
    ('left_knee_jc_3d','left_hip_jc_3d'),
    ('left_ankle_jc_3d','left_knee_jc_3d'),
    ('right_knee_jc_3d','right_hip_jc_3d'),
    ('right_ankle_jc_3d','right_knee_jc_3d'),
    ('left_elbow_jc_3d','left_shoulder_jc_3d'),
    ('right_elbow_jc_3d','right_shoulder_jc_3d'),
    ('left_elbow_jc_3d','left_wrist_jc_3d'),
    ('right_elbow_jc_3d','right_wrist_jc_3d'),
]

anim = Animated.Animate_3d(markers,speed,90,axis,name,240,connections)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()